In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.sql import SparkSession
from pyspark.mllib.classification import LabeledPoint
from pyspark.mllib.evaluation import BinaryClassificationMetrics


In [2]:
# when set local[*], all available cores will be used, the only executor is the driver
conf = SparkConf().setMaster("local[*]").setAppName("HIGGS") 
conf.set("spark.executor.memory", "4G")
conf.set("spark.driver.memory", "20G")
# conf.set("spark.executor.cores", "4")
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
# conf.set("spark.default.parallelism", "4")
# create a Spark Session instead of a Spark Context
spark = SparkSession.builder \
    .config(conf = conf) \
  .appName("spark session example") \
  .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/03 23:55:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.option("delimiter", ",").option("header", "false").csv('/work/li.baol/data/HIGGS.csv')
df.show()

22/12/03 23:56:07 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                 _c0|                 _c1|                 _c2|                 _c3|                 _c4|                 _c5|                 _c6|                 _c7|                 _c8|                 _c

In [4]:
data = df.rdd.map(lambda row: LabeledPoint(float(row['_c0']), list(map(lambda x: float(x), row.asDict().values()))[1:]))

In [5]:
data.take(2)

[LabeledPoint(1.0, [0.869293212890625,-0.6350818276405334,0.22569026052951813,0.327470064163208,-0.6899932026863098,0.7542022466659546,-0.24857313930988312,-1.0920639038085938,0.0,1.3749921321868896,-0.6536741852760315,0.9303491115570068,1.1074360609054565,1.138904333114624,-1.5781983137130737,-1.046985387802124,0.0,0.657929539680481,-0.010454569943249226,-0.0457671694457531,3.101961374282837,1.353760004043579,0.9795631170272827,0.978076159954071,0.9200048446655273,0.7216574549674988,0.9887509346008301,0.8766783475875854]),
 LabeledPoint(1.0, [0.9075421094894409,0.3291472792625427,0.3594118654727936,1.4979698657989502,-0.3130095303058624,1.09553062915802,-0.5575249195098877,-1.588229775428772,2.1730761528015137,0.8125811815261841,-0.2136419266462326,1.2710145711898804,2.214872121810913,0.4999939501285553,-1.2614318132400513,0.7321561574935913,0.0,0.39870089292526245,-1.138930082321167,-0.0008191101951524615,0.0,0.3022198975086212,0.8330481648445129,0.9856996536254883,0.9780983924865723

In [6]:
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [7]:
# trainingData.count()

In [8]:
model = DecisionTree.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                    impurity='gini', maxDepth=8)

In [9]:
# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
predictions

MapPartitionsRDD[59] at mapPartitions at PythonMLLibAPI.scala:1344

In [10]:
predictions.take(15), testData.map(lambda x: x.label).take(15)

([0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0],
 [0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0])

In [11]:
labels = testData.map(lambda lp: lp.label)
labels

PythonRDD[62] at RDD at PythonRDD.scala:53

In [14]:
predsAndLabels = testData.map(lambda lp: (model.predict(lp.features), lp.label))
# predsAndLabels.collect()
# labels = labels.distinct()
# labels.collect()

In [15]:
metrics = BinaryClassificationMetrics(predsAndLabels)

Traceback (most recent call last):
  File "/work/li.baol/spark-3.3.1-bin-hadoop2/python/pyspark/serializers.py", line 458, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/work/li.baol/spark-3.3.1-bin-hadoop2/python/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/work/li.baol/spark-3.3.1-bin-hadoop2/python/pyspark/cloudpickle/cloudpickle_fast.py", line 602, in dump
    return Pickler.dump(self, obj)
  File "/work/li.baol/spark-3.3.1-bin-hadoop2/python/pyspark/context.py", line 447, in __getnewargs__
    raise RuntimeError(
RuntimeError: It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.


PicklingError: Could not serialize object: RuntimeError: It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.

In [ ]:
filter_label = labels.zipWithIndex().filter(lambda x: x[1] < 100).keys()
filter_pred = predictions.zipWithIndex().filter(lambda x: x[1] < 100).keys()
predsAndLabels = filter_label.zip(filter_pred) #predictions.zip(labels)
predsAndLabels.collect()

In [ ]:
filter_label.glom().map(len).collect(), filter_pred.glom().map(len).collect()

# TODO: Allocate more cores, and see if it's possible to directly construct an RDD. Or try a different serializer

In [ ]:
filter_label.zip(filter_label).take(10)

In [ ]:
filter_pred.collect(), filter_label.collect()

In [ ]:
labels.count()

In [ ]:
predictions.count()

In [ ]:
predsAndLabels.count()

In [ ]:
# testMSE = predsAndLabels.map(lambda lp: (lp[1] - lp[0]) * (lp[1] - lp[0])).sum() /\
#     float(test_data.count())
metrics = BinaryClassificationMetrics(predsAndLabels)    
print(f'AUC = {metrics.areaUnderROC}')
